In [1]:
# Load packages and World Bank Data

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
pd.options.plotting.backend = "plotly"
%pip install wbdata
import wbdata
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='wbdata')

  Using cached wbdata-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached dateparser-1.2.0-py2.py3-none-any.whl.metadata (28 kB)
  Using cached shelved_cache-0.3.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached tabulate-0.8.10-py3-none-any.whl.metadata (25 kB)
  Using cached tzlocal-5.2-py3-none-any.whl.metadata (7.8 kB)
Using cached wbdata-1.0.0-py3-none-any.whl (18 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Using cached dateparser-1.2.0-py2.py3-none-any.whl (294 kB)
Using cached shelved_cache-0.3.1-py3-none-any.whl (7.8 kB)
Using cached tabulate-0.8.10-py3-none-any.whl (29 kB)
Using cached tzlocal-5.2-py3-none-any.whl (17 kB)
Note: you may need to restart the kernel to use updated packages.


Key '774735490529441264' not in persistent cache.
Key '-9157692132494410117' not in persistent cache.
Key '-3337778076955976450' not in persistent cache.
Key '-526774689438690483' not in persistent cache.
Key '7136414795544235951' not in persistent cache.
Key '8557971468098301490' not in persistent cache.
Key '-5169092351530701386' not in persistent cache.
Key '2494097133452225048' not in persistent cache.
Key '1529381217336825627' not in persistent cache.
Key '4208659525936938775' not in persistent cache.
Key '-306072962661297096' not in persistent cache.
Key '7029583251520518403' not in persistent cache.
Key '2812724743383719020' not in persistent cache.
Key '-235077018809511948' not in persistent cache.
Key '-4470102919927082806' not in persistent cache.
Key '-3762036792896484504' not in persistent cache.
Key '1435239183290266423' not in persistent cache.
Key '4868880650159235130' not in persistent cache.
Key '8133712752157793436' not in persistent cache.
Key '-1798714870866057945' 

In [2]:
# Set World Bank Data Sources/Indicators for Population
 
SOURCE = 40 # "Population estimates and projections
pop_indicators = wbdata.get_indicators(source=SOURCE)

pop_indicators

Key '261700270427160297' not in persistent cache.


id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.0514        Number of deaths ages 5-14 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1019        Number of deaths ages 10-19 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.0514        Probability of dying at age 5-14 years (per 1,000 children age 5)
SH.DYN.1014        Probability of dying among adolescents ages 1

In [3]:
SAcountries = {"BGD":"Bangladesh",
             "BTN":"Bhutan",
             "IND": "India",
             "MDV": "Maldives",
             "NPL": "Nepal",
             "PAK": "Pakistan",
             "LKA": "Sri Lanka",
             "TSA":"South Asia (IDA & IBRD)"
            }
def get_population_by_age_sex(countries):
    """
    Fetches population data by age and sex from the World Bank (source 40)
    and returns a pandas DataFrame indexed by Country/Region and Year.
    
    :param countries: List of country ISO codes or 'all' for all available.
    :param years: List of years as integers.
    :return: Pandas DataFrame with population counts by age and sex.
    """
    # Define indicators for population by age and sex (source 40)
    indicators = {
        "SP.POP.0004.MA": "Male 0-4",
        "SP.POP.0004.FE": "Female 0-4",
        "SP.POP.0509.MA": "Male 5-9",
        "SP.POP.0509.FE": "Female 5-9",
        "SP.POP.1014.MA": "Male 10-14",
        "SP.POP.1014.FE": "Female 10-14",
        "SP.POP.1519.MA": "Male 15-19",
        "SP.POP.1519.FE": "Female 15-19",
        "SP.POP.2024.MA": "Male 20-24",
        "SP.POP.2024.FE": "Female 20-24",
        "SP.POP.2529.MA": "Male 25-29",
        "SP.POP.2529.FE": "Female 25-29",
        "SP.POP.3034.MA": "Male 30-34",
        "SP.POP.3034.FE": "Female 30-34",
        "SP.POP.3539.MA": "Male 35-39",
        "SP.POP.3539.FE": "Female 35-39",
        "SP.POP.4044.MA": "Male 40-44",
        "SP.POP.4044.FE": "Female 40-44",
        "SP.POP.4549.MA": "Male 45-49",
        "SP.POP.4549.FE": "Female 45-49",
        "SP.POP.5054.MA": "Male 50-54",
        "SP.POP.5054.FE": "Female 50-54",
        "SP.POP.5559.MA": "Male 55-59",
        "SP.POP.5559.FE": "Female 55-59",
        "SP.POP.6064.MA": "Male 60-64",
        "SP.POP.6064.FE": "Female 60-64",
        "SP.POP.6569.MA": "Male 65-69",
        "SP.POP.6569.FE": "Female 65-69",
        "SP.POP.7074.MA": "Male 70-74",
        "SP.POP.7074.FE": "Female 70-74",
        "SP.POP.7579.MA": "Male 75-79",
        "SP.POP.7579.FE": "Female 75-79",
        "SP.POP.80UP.MA": "Male 80+",
        "SP.POP.80UP.FE": "Female 80+"
    }
    
    # Convert years to string format    
    # Fetch data from World Bank API
    df = wbdata.get_dataframe(indicators, country=countries,  parse_dates=True).squeeze()
    df = df.rename(columns={'date': 'Date'})
    
    return df

get_population_by_age_sex(SAcountries)

Male 0-4  Female 0-4    Male 5-9  \
country                 date                                             
Bangladesh              2023-01-01   8451170.0   8053899.0   8089646.0   
                        2022-01-01   8349752.0   7941181.0   8144991.0   
                        2021-01-01   8252715.0   7834808.0   8187045.0   
                        2020-01-01   8166737.0   7746950.0   8179783.0   
                        2019-01-01   8085218.0   7672802.0   8154366.0   
...                                        ...         ...         ...   
South Asia (IDA & IBRD) 1964-01-01  52856156.0  50169637.0  43641688.0   
                        1963-01-01  51633386.0  49002230.0  42529001.0   
                        1962-01-01  50444081.0  47872703.0  41351758.0   
                        1961-01-01  49281829.0  46773844.0  40114559.0   
                        1960-01-01  48137844.0  45700939.0  38840109.0   

                                    Female 5-9  Male 10-14  Female 10-14  \
country                 date                                               
Bangladesh              2023-01-01   7586510.0   8594037.0     7854290.0   
                        2022-01-01   7586974.0   8702787.0     8006502.0   
                        2021-01-01   7598439.0   8716928.0     8156041.0   
                        2020-01-01   7613840.0   8664017.0     8284906.0   
                        2019-01-01   7652839.0   8644445.0     8400369.0   
...                                        ...         ...           ...   
South Asia (IDA & IBRD) 1964-01-01  40904868.0  37013041.0    34550927.0   
                        1963-01-01  39870765.0  35507401.0    33169464.0   
                        1962-01-01  38780424.0  33891645.0    31752350.0   
                        1961-01-01  37636034.0  32429523.0    30515007.0   
                        1960-01-01  36454036.0  31105957.0    29396110.0   

                                    Male 15-19  Female 15-19  Male 20-24  \
country                 date                                               
Bangladesh              2023-01-01   8535908.0     8609733.0   7295364.0   
                        2022-01-01   8404758.0     8731548.0   7179638.0   
                        2021-01-01   8284911.0     8772030.0   7232440.0   
                        2020-01-01   8253041.0     8748000.0   7395616.0   
                        2019-01-01   8315398.0     8709213.0   7497693.0   
...                                        ...           ...         ...   
South Asia (IDA & IBRD) 1964-01-01  29445290.0    27744752.0  25843619.0   
                        1963-01-01  28601144.0    27031186.0  25482311.0   
                        1962-01-01  28019908.0    26495065.0  25187056.0   
                        1961-01-01  27443686.0    25936512.0  24894325.0   
                        1960-01-01  26896563.0    25428963.0  24566431.0   

                                    Female 20-24  ...  Male 60-64  \
country                 date                      ...               
Bangladesh              2023-01-01     8600067.0  ...   2650737.0   
                        2022-01-01     8587964.0  ...   2595789.0   
                        2021-01-01     8536115.0  ...   2557008.0   
                        2020-01-01     8402872.0  ...   2523977.0   
                        2019-01-01     8198837.0  ...   2473877.0   
...                                          ...  ...         ...   
South Asia (IDA & IBRD) 1964-01-01    24286307.0  ...   7105070.0   
                        1963-01-01    23987884.0  ...   6962203.0   
                        1962-01-01    23764048.0  ...   6840890.0   
                        1961-01-01    23540055.0  ...   6696640.0   
                        1960-01-01    23269837.0  ...   6503546.0   

                                    Female 60-64  Male 65-69  Female 65-69  \
country                 date                                                 
Bangladesh              2023-01-01     2441